## ERUPT under simulated random assignment

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))

from causaltune import CausalTune
from causaltune.datasets import generate_non_random_dataset
from causaltune.erupt import DummyPropensity, ERUPT


In [ ]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%javascript

// turn off scrollable windows for large output
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Loading data and model training

In [ ]:
# load toy dataset with non-random assignment and apply standard pre-processing
cd = generate_non_random_dataset()
cd.preprocess_dataset()

In [ ]:
display(cd.data.head())

In [ ]:
# training configs

# set evaluation metric
metric = "energy_distance"

# it's best to specify either time_budget or components_time_budget, 
# and let the other one be inferred; time in seconds
time_budget = None
components_time_budget = 10

# specify training set size
train_size = 0.7

In [ ]:
ct = CausalTune(
    estimator_list='cheap_inference',
    metric=metric,
    verbose=0,
    components_verbose=0,
    time_budget=time_budget,
    components_time_budget=components_time_budget,
    train_size=train_size
)


# run causaltune
ct.fit(data=cd, outcome=cd.outcomes[0])

print('---------------------')
# return best estimator
print(f"Best estimator: {ct.best_estimator}")
# config of best estimator:
print(f"Best config: {ct.best_config}")
# best score:
print(f"Best score: {ct.best_score}")

## Random ERUPT

Below we demonstrate how to use Estimated Response Under Proposed Treatment (ERUPT) to estimate the average treatment effect had the treatment been assigned randomly. Recall that the dataset used in this example is constructed in a way that the treatment propensity is a function of a unit's covariates.

In [ ]:
use_df = ct.test_df

In [ ]:
# computing mean ERUPT over 10 bootstrapped samples

scores_list = []

for i in range(10):

    bootstrap_df = use_df.sample(frac=1, replace=True)
    propensities = bootstrap_df['propensity']
    actual_treatment = bootstrap_df['T']
    outcome = bootstrap_df['Y']

    # define the random assignment policy
    random_policy = np.random.randint(0,2, size=len(bootstrap_df))

    # define a propensity model that will simply return the propensities when calling predict_proba
    propensity_model = DummyPropensity(p=propensities, treatment=actual_treatment)

    # obtain ERUPT under random policy
    e = ERUPT(treatment_name='T', propensity_model=propensity_model)
    scores_list.append(e.score(df=use_df,outcome=outcome,policy=random_policy))

erupt_mean = np.mean(scores_list)
erupt_sd = np.std(scores_list)

In [ ]:
# compute naive ate as difference in means
naive_ate, naive_sd, _ = ct.scorer.naive_ate(ct.test_df['T'], ct.test_df['Y'])

In [ ]:
# comparison of naive ate to mean random erupt over 10 bootstrap runs
erupt_df = pd.DataFrame([[naive_ate,naive_sd],[erupt_mean,erupt_sd]], columns=['estimated_effect', 'sd'], index=['naive_ate','random_erupt'])
display(erupt_df)

For more details on the ERUPT implementation, consult [Hitsch and Misra (2018)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3111957). Note also that we assume that treatment takes integer values from 0 to n.